In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')
data

In [ ]:
def down_sizing(X):
    X.assists=X.assists.astype('int8')
    X.boosts=X.boosts.astype('int8')
    X.DBNOs=X.DBNOs.astype('int8')
    X.headshotKills=X.headshotKills.astype('int8')
    X.heals=X.heals.astype('int8')
    X.killPlace=X.killPlace.astype('int8')
    X.killPoints=X.killPoints.astype('int16')
    X.kills=X.kills.astype('int8')
    X.killStreaks=X.killStreaks.astype('int8')
    X.matchDuration=X.matchDuration.astype('int16')
    X.maxPlace=X.maxPlace.astype('int8')
    X.numGroups=X.numGroups.astype('int8')
    X.rankPoints=X.rankPoints.astype('int16')
    X.revives=X.revives.astype('int8')
    X.roadKills=X.roadKills.astype('int8')
    X.teamKills=X.teamKills.astype('int8')
    X.vehicleDestroys=X.vehicleDestroys.astype('int8')
    X.weaponsAcquired=X.weaponsAcquired.astype('int16')
    X.winPoints=X.winPoints.astype('int16')

In [ ]:
down_sizing(data)

In [ ]:
matches = data.matchId.unique()


In [ ]:
training_matches = matches[:int(len(matches)*80//100)]
validation_matches = matches[int(len(matches)*80//100):]

In [ ]:
len(matches)*80//100 + len(matches)*20//100

In [ ]:
data_train = data[data['matchId'].isin(training_matches)]

In [ ]:
data_validation = data[data['matchId'].isin(validation_matches)]

In [ ]:
data_train.head()

In [ ]:
data.info()

In [ ]:
data_train.columns.shape

In [ ]:
data_train.isna().sum()

In [ ]:
data_train_labels = data_train['winPlacePerc']

In [ ]:
data_validation_labels = data_validation['winPlacePerc']

In [ ]:
data_validation.drop(['winPlacePerc'] , axis =1 , inplace=True)

In [ ]:
data_train.drop(['winPlacePerc'] , axis =1 , inplace=True)

In [ ]:
data_train.columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
categorical_encoder = OneHotEncoder(sparse=False)
scaller = StandardScaler()

In [ ]:
data_categorical = data_train['matchType']
data_categorical

In [ ]:
data_categorical = data_train[['matchType']]
def data_encoder(X):
    data_categorical = X[['matchType']]
    X.drop(['matchId' , 'groupId' , 'Id'] , axis=1 , inplace=True)
    X['headshots_per_kills'] = X['headshotKills'] / X['kills']
    X['walk_per_second']=X['walkDistance']/X['matchDuration']
    X['kill_streak_rate'] = X['killStreaks'] / (X['kills'] + 0.00001)
    X['killPlace_over_maxPlace'] = X['killPlace'] / X['maxPlace']
    X['killPlace_over_maxPlace'].fillna(0, inplace=True)
    X['killPlace_over_maxPlace'].replace(np.inf, 0, inplace=True)
    X.fillna(0,inplace=True)
    X['whole_distance'] = X['rideDistance'] + X['swimDistance'] + X['walkDistance']
    X['Whole_distance_per_second']=X['whole_distance']/X['matchDuration']
    X["points_sum"] = X["killPoints"] + X["rankPoints"] + X["winPoints"]
    X.drop(['killPoints'] , axis = 1 , inplace=True)
    X.drop(['rankPoints'] , axis = 1 , inplace=True)
    X.drop(['winPoints'] , axis = 1 , inplace=True)
    X.drop(['rideDistance'] , axis = 1 , inplace=True)
    X.drop(['swimDistance'] , axis = 1 , inplace=True)
    X.drop(['matchType'] , axis = 1 , inplace=True)
    a = categorical_encoder.fit_transform(data_categorical)
    array = np.c_[X.values , a]
    final_array=scaller.fit_transform(array)
    return final_array


In [ ]:
data_train_2 = data_encoder(data_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 40, random_state = 42)
rf.fit(data_train_2, data_train_labels);

In [ ]:
aaaa = rf.predict(data_train_2)

In [ ]:
from sklearn.metrics import mean_absolute_error
error = mean_absolute_error(aaaa , data_train_labels)

In [ ]:
error

In [ ]:
del data_train_2
del data_train
gc.collect()

In [ ]:
data_validation_encoded = data_encoder(data_validation)


In [ ]:
data_validation_labels.fillna(0 , inplace=True)

In [ ]:
val=rf.predict(data_validation_encoded)

In [ ]:
validation_error = mean_absolute_error(val , data_validation_labels)

In [ ]:
del data_validation
del data_validation_encoded
del data
gc.collect()

In [ ]:
test = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
tset=down_sizing(test)

In [ ]:
id = test.Id

In [ ]:
test_encoded = data_encoder(test)

In [ ]:
submission1 = rf.predict(test_encoded)

In [ ]:
submission = np.c_[id,submission1]

In [ ]:
submission=pd.DataFrame(submission)

In [ ]:
submission.rename(columns={0:'Id' , 1 : 'winPlacePerc'} , inplace=True)
submission

In [ ]:
submission.to_csv('submission.csv' , index=False)